In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

plt.rcParams['figure.figsize'] = 12.0, 8.0

from pyts.transformation import GADF,GASF
from sklearn.preprocessing import normalize

import uproot

import keras
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Concatenate, Dense, Activation, Reshape, Conv1D , Conv2D, MaxPooling1D, MaxPooling2D
from keras.layers import Dropout, Flatten, BatchNormalization, Embedding, LSTM, UpSampling1D

from keras.models import load_model

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def GAN():
    police = load_model("police.h5")
    thief = load_model("thief.h5")
    police.name = "police"
    thief.name = "thief"
    police.summary()
    thief.summary()
    gan = Sequential()
    gan.add(thief)
    police.trainable = False
    gan.add(police)
    gan.summary()
    return gan

gan = GAN()
China = load_model("thief_3.h5")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4096)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 4096, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 128, 16)           1040      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4, 8)              8200      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 2, 8)              0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1, 8)              4104      
__________

In [3]:


classes = 7
group = 5
training_ratio = 0.7
validation_ratio = 0.1
test_ratio = 0.2

def label(q,k):
    x = np.zeros(len(q))
    for i in range(len(q)):
        x[i] = k
    return x

def sep(q,k,z):
    y = label(q,k)
    x1, x2 ,x3 = np.split(q,[int(len(q)*training_ratio),int(len(q)*(training_ratio+validation_ratio))])
    y1, y2 ,y3 = np.split(y,[int(len(q)*training_ratio),int(len(q)*(training_ratio+validation_ratio))])
    if z == 0:
        return x1, y1
    if z == 1:
        return x2, y2
    if z == 2:
        return x3, y3
    
def comb(one,two,three,four,five,portion):
    one1,one2 = sep(one,0,portion)
    two1,two2 = sep(two,1,portion)
    three1,three2 = sep(three,2,portion)
    four1,four2 = sep(four,3,portion)
    five1,five2 = sep(five,4,portion)
#     six1,six2 = sep(six,5,portion)
#     seven1,seven2 = sep(seven,6,portion)
    
    z = np.concatenate((one1,two1,three1,four1,five1),axis=0)
    y = np.concatenate((one2,two2,three2,four2,five2),axis=0)
    return z, y

def reader_pmtall(path):
    extra = np.arange(4096, 4480)
    
    tree = uproot.open(path)["tree"]
    pmtall = tree.array("PMTALL")
    pmtall = np.delete(pmtall, extra, axis=1)
    return pmtall

def reader(path,branch,number):
    tree = uproot.open(path)["tree"]
    column = tree.array(branch)
    column = column[:,number]
    return column

def reader_lone(path,branch):
    tree = uproot.open(path)["tree"]
    column = tree.array(branch)
    return column

def pmtall_pedestal(path):
    pedestal = reader(path,"Pedestal",0)
    pmtall = reader_pmtall(path)
    for i in range(len(pedestal)):
        pmtall[i] = -(pmtall[i]-pedestal[i])
    
    return pmtall

def generator(x):
    while 1:
        alpha = pmtall_pedestal("Alpha.root")
#         bad1 = pmtall_pedestal("BAD_DATA_QUALITY_1.root")
#         bad2 = pmtall_pedestal("BAD_DATA_QUALITY_2.root")
        piled = pmtall_pedestal("Piled_2.root")
        ls = pmtall_pedestal("LS.root")
        lsbeta = pmtall_pedestal("lsbeta.root")
        ref = pmtall_pedestal("RefPulse009.root")
        
        z1, z2 = comb(piled,ref,alpha,ls,lsbeta,x)
        s = np.random.permutation(np.arange(len(z2)))
        z1 = z1[s]
        z2 = z2[s]
        
#         print(len(z1))

#         para_piled = parameters("Piled_2.root")
#         para_ref = parameters("RefPulse009.root")
#         para_alpha = parameters("Alpha.root")
#         para_ls = parameters("LS.root")
#         para_lsbeta = parameters("lsbeta.root")
        
#         z3, z4 = comb5(para_piled,para_ref,para_alpha,para_ls,para_lsbeta,x)        
#         z3 = z3[s] 
#         z4 = z4[s]

        gasf = GASF(image_size=128, overlapping=False, scale='-1')
        
        for i in range(len(z2)):
            d1 = np.expand_dims(z1[i],axis=0)
            q = keras.utils.to_categorical(z2[i],classes)
            r = normalize(d1,norm="l1")
#             s = np.expand_dims(q,axis=0)
            yield r

test = generator(0)
# print(next(test)[0][0])
print(next(test)[0].shape)
# print(next(test)[0][1].shape)
# print(next(test)[1].shape)

(4096,)


In [4]:
%time
predicted = gan.predict_generator(generator(0),max_queue_size=100,steps=2000,verbose=1)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 10 µs
2000/2000 [==============================] - 45s 22ms/step


In [5]:
print(predicted[:100])
print(predicted.argmax(axis=1)[:100])

[[0.0000000e+00 1.0000000e+00]
 [6.0132095e-03 9.9351370e-01]
 [1.7049118e-20 1.0000000e+00]
 [6.8724073e-02 9.3106544e-01]
 [6.2236553e-38 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [5.4902355e-03 9.9405730e-01]
 [1.8015896e-03 9.9796617e-01]
 [5.8859102e-03 9.9364591e-01]
 [1.1111457e-01 8.9002836e-01]
 [2.8418176e-02 9.7062868e-01]
 [0.0000000e+00 1.0000000e+00]
 [2.5301626e-02 9.7374874e-01]
 [0.0000000e+00 1.0000000e+00]
 [3.2060966e-02 9.6699601e-01]
 [0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [2.9351428e-02 9.6969664e-01]
 [0.0000000e+00 1.0000000e+00]
 [1.3601923e-35 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.2235206e-02 9.8706472e-01]
 [4.6697825e-02 9.5253271e-01]
 [5.8986005e-38 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.8240298e-36 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.3793886e-02 9.8546350e-01]
 [1.4276967e-02 9.8496789e-01]
 [2.3544985e-36 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [4.5938222e-03 9.9499387e-01]
 [7.6199

In [4]:
def standard(x,height,decay):
    y = height*np.exp(-x*decay)
    return y

def shifter(x,starts):
    L = int(starts*len(x))
    y = np.zeros(len(x))
    y[:L] = np.zeros(L) 
    y[L:] = x[:(len(x)-L)]
    return y

def comb_standard(x,second,height_1,decay_1,height_2,decay_2):
    L = int(second*len(x))
    y = np.zeros(len(x))
    y[:L] = standard(x[:L],height_1,decay_1)
    y[L:] = standard(x[:(len(x)-L)],height_2,decay_2)
    return y

def noiser(x,strength):
    y = x + np.random.normal(0,strength,len(x))
    return y

def noiser_long(x,strength):
    noise = np.random.normal(0,strength,len(x))
    y = x + np.cumsum(noise)*strength
    return y

def noiser_comb(x,sepfact,strength):
    L = int(sepfact*len(x))
    x[:L] = noiser(x[:L],strength)
    x[L:] = noiser_long(x[L:],strength)
    return x

def array_maker(entries):
    x = np.arange(0,1,1/4096)
    x = np.expand_dims(x,axis=0)
    x = np.tile(x,[entries,1])
    return x

def double(x,second,height_1,decay_1,height_2,decay_2,starts,sepfact=0.15,strength=0.02):
    y = comb_standard(x,second,height_1,decay_1,height_2,decay_2)
    y = shifter(y,starts)
    y = noiser_comb(y,sepfact,strength)
    return y

def single(x,height,decay,starts,sepfact=0.15,strength=0.02):
    y = standard(x,height,decay)
    y = shifter(y,starts)
    y = noiser_comb(y,sepfact,strength)
    return y

def event_creators_single(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(5,2)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_single_2(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(10,2)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_single_3(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.1)
        r = np.random.normal(4,1)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_sharp(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(100,20)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)    
    return x

def event_creators_double_equal(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.2,0.01)
        p = np.random.normal(1,0.01)
        q = np.random.normal(5,2)
        r = np.random.normal(1,0.01)
        s = np.random.normal(5,2)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x
    
def event_creators_double_unequal(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.2,0.01)
        p = np.random.normal(1,0.2)
        q = np.random.normal(50,10)
        r = np.random.normal(1,0.2)
        s = np.random.normal(50,10)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x

def event_creators_sharp_fat(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.015,0.001)
        p = np.random.normal(1,0.2)
        q = np.random.normal(100,20)
        r = np.random.normal(0.2,0.05)
        s = np.random.normal(5,2)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x

In [5]:
def gen_fake(x):
    while 1:
        beta = event_creators_single(1000)
        ls = event_creators_sharp(1000)
        dp_e = event_creators_double_equal(1000)
        dp_ue = event_creators_double_unequal(1000)
        lsbeta = event_creators_sharp_fat(1000)
        
        alpha = event_creators_single_2(1000)
        gamma = event_creators_single_3(1000)
        
        z1, z2 = comb(beta,ls,dp_e,alpha,gamma,x)
        s = np.random.permutation(np.arange(len(z2)))
        z1 = z1[s]
        z2 = z2[s]
#         print(len(z1))
        gasf = GASF(image_size=128, overlapping=False, scale='-1')
        
        for i in range(len(z2)):
            d1 = np.expand_dims(z1[i],axis=0)
#             p = gasf.transform(d1)
            q = keras.utils.to_categorical(z2[i],classes)
            r = normalize(d1,norm="l1")
            yield r,r



In [8]:
%time
predicted = gan.predict_generator(gen_fake(0),max_queue_size=100,steps=2000,verbose=1)
print(predicted[:100])
print(predicted.argmax(axis=1)[:100])

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs
2000/2000 [==============================] - 24s 12ms/step
[[1.02349564e-01 8.98571908e-01]
 [2.20375471e-02 9.77042437e-01]
 [1.12340860e-01 8.88832748e-01]
 [8.41880776e-03 9.91010427e-01]
 [1.32688852e-02 9.86002207e-01]
 [3.46793048e-02 9.64393497e-01]
 [7.33393654e-02 9.26587999e-01]
 [7.87565578e-03 9.91575301e-01]
 [1.00000000e+00 8.99919896e-12]
 [9.99999642e-01 2.39477231e-07]
 [1.00000000e+00 4.58177782e-15]
 [1.67162605e-02 9.82466459e-01]
 [1.37920156e-01 8.63518417e-01]
 [6.34470861e-03 9.93170142e-01]
 [3.38822566e-02 9.65185106e-01]
 [1.00000000e+00 4.32321483e-14]
 [2.23504975e-02 9.76725042e-01]
 [1.73934594e-01 8.27491343e-01]
 [1.14765670e-02 9.87846076e-01]
 [1.49155855e-01 8.52354586e-01]
 [6.98595308e-03 9.92503405e-01]
 [1.00000000e+00 5.16984167e-21]
 [7.29603589e-01 2.72404015e-01]
 [1.18273795e-02 9.87484515e-01]
 [2.15070963e-01 7.86692202e-01]
 [8.96255020e-03 9.90445733e-01]
 [1.00000000e+00 

In [6]:
%time
def comb1(one,portion):
    one1,one2 = sep(one,0,portion)
#     two1,two2 = sep(two,1,portion)
#     three1,three2 = sep(three,2,portion)
#     four1,four2 = sep(four,3,portion)
#     five1,five2 = sep(five,4,portion)
#     six1,six2 = sep(six,5,portion)
#     seven1,seven2 = sep(seven,6,portion)
    
#     z = np.concatenate((one1),axis=0)
#     y = np.concatenate((one2),axis=0)
    return one1,one2

def single_gen(x):
    while 1:
#         alpha = pmtall_pedestal("Alpha.root")
#         bad1 = pmtall_pedestal("BAD_DATA_QUALITY_1.root")
#         bad2 = pmtall_pedestal("BAD_DATA_QUALITY_2.root")
        piled = pmtall_pedestal("Piled_2.root")
#         ls = pmtall_pedestal("LS.root")
#         lsbeta = pmtall_pedestal("lsbeta.root")
#         ref = pmtall_pedestal("RefPulse009.root")
        
        z1, z2 = comb1(piled,x)
        s = np.random.permutation(np.arange(len(z2)))
        z1 = z1[s]
        z2 = z2[s]
        
#         print(len(z1))

#         para_piled = parameters("Piled_2.root")
#         para_ref = parameters("RefPulse009.root")
#         para_alpha = parameters("Alpha.root")
#         para_ls = parameters("LS.root")
#         para_lsbeta = parameters("lsbeta.root")
        
#         z3, z4 = comb5(para_piled,para_ref,para_alpha,para_ls,para_lsbeta,x)        
#         z3 = z3[s] 
#         z4 = z4[s]

        gasf = GASF(image_size=128, overlapping=False, scale='-1')
        
        for i in range(len(z2)):
            d1 = np.expand_dims(z1[i],axis=0)
            q = keras.utils.to_categorical(z2[i],classes)
            r = normalize(d1,norm="l1")
#             s = np.expand_dims(q,axis=0)
            yield r


# predicted = China.predict_generator(generator(0),max_queue_size=100,steps=2000,verbose=1)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.06 µs


In [ ]:
for i in range(20):
    X1 = next(generator(0))
    print(X1.shape)
    X2 = China.predict(X1,verbose=1)
    print(X2)
    plt.figure()
    plt.plot(X1[0],alpha=1)
    plt.plot(X2[0],alpha=0.7)
    plt.show()
    

In [5]:
classes = 2

def standard(x,height,decay):
    y = height*np.exp(-x*decay)
    return y

def shifter(x,starts):
    L = int(starts*len(x))
    y = np.zeros(len(x))
    y[:L] = np.zeros(L) 
    y[L:] = x[:(len(x)-L)]
    return y

def comb_standard(x,second,height_1,decay_1,height_2,decay_2):
    L = int(second*len(x))
    y = np.zeros(len(x))
    y[:L] = standard(x[:L],height_1,decay_1)
    y[L:] = standard(x[:(len(x)-L)],height_2,decay_2)
    return y

def noiser(x,strength):
    y = x + np.random.normal(0,strength,len(x))
    return y

def noiser_long(x,strength):
    noise = np.random.normal(0,strength,len(x))
    y = x + np.cumsum(noise)*strength
    return y

def noiser_comb(x,sepfact,strength):
    L = int(sepfact*len(x))
    x[:L] = noiser(x[:L],strength)
    x[L:] = noiser_long(x[L:],strength)
    return x

def array_maker(entries):
    x = np.arange(0,1,1/4096)
    x = np.expand_dims(x,axis=0)
    x = np.tile(x,[entries,1])
    return x

def double(x,second,height_1,decay_1,height_2,decay_2,starts,sepfact=0.15,strength=0.02):
    y = comb_standard(x,second,height_1,decay_1,height_2,decay_2)
    y = shifter(y,starts)
    y = noiser_comb(y,sepfact,strength)
    return y

def single(x,height,decay,starts,sepfact=0.15,strength=0.02):
    y = standard(x,height,decay)
    y = shifter(y,starts)
    y = noiser_comb(y,sepfact,strength)
    return y

def event_creators_single(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(5,2)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_single_2(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(10,2)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_single_3(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.1)
        r = np.random.normal(4,1)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)
    return x

def event_creators_sharp(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(1,0.01)
        r = np.random.normal(100,20)
        s = np.random.normal(0.03,0.005)
        x[i] = single(x[i],w,r,s)    
    return x

def event_creators_double_equal(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.2,0.01)
        p = np.random.normal(1,0.01)
        q = np.random.normal(5,2)
        r = np.random.normal(1,0.01)
        s = np.random.normal(5,2)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x
    
def event_creators_double_unequal(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.2,0.01)
        p = np.random.normal(1,0.2)
        q = np.random.normal(50,10)
        r = np.random.normal(1,0.2)
        s = np.random.normal(50,10)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x

def event_creators_sharp_fat(entries):
    x = array_maker(entries)
    for i in range(entries):
        w = np.random.normal(0.015,0.001)
        p = np.random.normal(1,0.2)
        q = np.random.normal(100,20)
        r = np.random.normal(0.2,0.05)
        s = np.random.normal(5,2)
        t = np.random.normal(0.03,0.005)
        x[i] = double(x[i],w,p,q,r,s,t)
    return x

def comb(one,two,three,four,five,six,seven,eight,nine,ten,portion):
    one1,one2 = sep(one,1,portion)
    two1,two2 = sep(two,1,portion)
    three1,three2 = sep(three,1,portion)
    four1,four2 = sep(four,1,portion)
    five1,five2 = sep(five,1,portion)
    six1,six2 = sep(six,0,portion)
    seven1,seven2 = sep(seven,0,portion)
    eight1,eight2 = sep(eight,0,portion)
    nine1,nine2 = sep(nine,0,portion)
    ten1,ten2 = sep(ten,0,portion)

    z = np.concatenate((one1,two1,three1,four1,five1,six1,seven1,eight1,nine1,ten1),axis=0)
    y = np.concatenate((one2,two2,three2,four2,five2,six2,seven2,eight2,nine2,ten2),axis=0)
    return z, y

def generator(x):
    while 1:
        alpha = pmtall_pedestal("Alpha.root")
#         bad1 = pmtall_pedestal("BAD_DATA_QUALITY_1.root")
#         bad2 = pmtall_pedestal("BAD_DATA_QUALITY_2.root")
        piled = pmtall_pedestal("Piled_2.root")
        ls = pmtall_pedestal("LS.root")
        lsbeta = pmtall_pedestal("lsbeta.root")
        ref = pmtall_pedestal("RefPulse009.root")
        
        fake_beta = event_creators_single(4000)
        fake_ls = event_creators_sharp(4000)
        fake_dp_e = event_creators_double_equal(4000)
        fake_dp_ue = event_creators_double_unequal(4000)
        fake_lsbeta = event_creators_sharp_fat(4000)
        
        z1, z2 = comb(piled,ref,alpha,ls,lsbeta,fake_beta,fake_ls,fake_dp_e,fake_dp_ue,fake_lsbeta,x)
        s = np.random.permutation(np.arange(len(z2)))
        z1 = z1[s]
        z2 = z2[s]
#         print(len(z1))
#         para_piled = parameters("Piled_2.root")
#         para_ref = parameters("RefPulse009.root")
#         para_alpha = parameters("Alpha.root")
#         para_ls = parameters("LS.root")
#         para_lsbeta = parameters("lsbeta.root")
        
#         z3, z4 = comb5(para_piled,para_ref,para_alpha,para_ls,para_lsbeta,x)        
#         z3 = z3[s] 
#         z4 = z4[s]

        gasf = GASF(image_size=128, overlapping=False, scale='-1')
        
        for i in range(len(z2)):
            d1 = np.expand_dims(z1[i],axis=0)
            q = keras.utils.to_categorical(z2[i],classes)
            r = normalize(d1,norm="l1")
            s = np.expand_dims(q,axis=0)
            yield r,s
            
train_gen = generator(0)
val_gen = generator(1)
test_gen = generator(2)

steps_t = 35430
steps_vt = 5057
gan.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics=['accuracy'])
gan.fit_generator(train_gen,steps_per_epoch=steps_t, epochs=2, verbose=1, validation_data=val_gen,validation_steps=steps_vt)

Epoch 1/2
35430/35430 [==============================] - 2313s 65ms/step - loss: 3.9119 - acc: 0.3952 - val_loss: 3.9103 - val_acc: 0.3945
Epoch 2/2
35430/35430 [==============================] - 2215s 63ms/step - loss: 3.9105 - acc: 0.3951 - val_loss: 3.9103 - val_acc: 0.3945


In [8]:
gan.fit_generator(train_gen,steps_per_epoch=steps_t, epochs=5, verbose=1, validation_data=val_gen,validation_steps=steps_vt)
gan.save("retrained-GAN-freeze-police.h5")

Epoch 1/5
20752/35430 [================>.............] - ETA: 15:14 - loss: 3.9154 - acc: 0.3936

KeyboardInterrupt: 

In [7]:
predicted = gan.predict_generator(generator(0),max_queue_size=100,steps=2000,verbose=1)

2000/2000 [==============================] - 52s 26ms/step


array([[0.99836427, 0.00150627],
       [0.99836427, 0.00150627],
       [0.99836427, 0.00150627],
       ...,
       [0.99836427, 0.00150627],
       [0.99836427, 0.00150627],
       [0.99836427, 0.00150627]], dtype=float32)